Author do not assume and hereby disclaim any liability to any party for any loss, damage, or disruption caused by errors or omissions, whether such errors or omissions result from negligence, accident, or any other cause.
The software is provided "as is", without warranty of any kind, express or implied, including but not limited to the warranties of merchantability, fitness for a particular purpose and noninfringement.

In [1]:
"""
REFERENCES

https://www.anaconda.com/download/
http://jupyter.org/
https://www.python.org/

https://www.anaf.ro/anaf/internet/ANAF/informatii_publice/informatii_agenti_economici/RegPlataDefalcataTVA
https://webservicesp.anaf.ro/AsynchWebService/api/v3/ws/tva

Warning in Romanian language
b) Orice tentativa de suprasolicitare a serverului va fi pedepsita conform reglementarilor in vigoare.
Warning in English language
b) Any attempt to overload the server will be punished according to the regulations in force.
"""

'\nREFERENCES\n\nhttps://www.anaconda.com/download/\nhttp://jupyter.org/\nhttps://www.python.org/\n\nhttps://www.anaf.ro/anaf/internet/ANAF/informatii_publice/informatii_agenti_economici/RegPlataDefalcataTVA\nhttps://webservicesp.anaf.ro/AsynchWebService/api/v3/ws/tva\n\nWarning in Romanian language\nb) Orice tentativa de suprasolicitare a serverului va fi pedepsita conform reglementarilor in vigoare.\nWarning in English language\nb) Any attempt to overload the server will be punished according to the regulations in force.\n'

In [2]:
"""
CREATE INPUT FILE IN CSV FORMAT

Source path_upload: active_vendor.csv:
Vendor_key,Vendor_name,CUI
"SNN","S.N. NUCLEARELECTRICA S.A.","10874881"
"SNP","OMV PETROM S.A.","1590082"
"""

'\nCREATE INPUT FILE IN CSV FORMAT\n\nSource path_upload: active_vendor.csv:\nVendor_key,Vendor_name,CUI\n"SNN","S.N. NUCLEARELECTRICA S.A.","10874881"\n"SNP","OMV PETROM S.A.","1590082"\n'

In [3]:
# Run cell
date = '2018-02-12' # query date
path_upload = "C:/00SplitVAT/active_vendor.csv"
path_save = "C:/00SplitVAT/active_vendor_result.csv"

In [4]:
# Run cell
import requests
import json
import timeit
import time
import csv
import sys

print(sys.version)
# https://www.anaf.ro/anaf/internet/ANAF/informatii_publice/informatii_agenti_economici/RegPlataDefalcataTVA
# url = "https://webservicesp.anaf.ro/AsynchWebService/api/v3/ws/tva" # Asynchronous Web Service
url = "https://webservicesp.anaf.ro/PlatitorTvaRest/api/v3/ws/tva" # Synchronous Web Service

file_upload = open(path_upload, newline='')
reader = csv.reader(file_upload)
header = next(reader)

file_save = open(path_save, 'w', encoding="utf-8")
writer = csv.writer(file_save)

writer.writerow (["Vendor_key", "Vendor_name", "CUI", "status_code", "data_set", \
                 "cui", "data", "denumire", "adresa", "scpTVA", "data_inceput_ScpTVA", \
                  "data_sfarsit_ScpTVA", "data_anul_imp_ScpTVA", "mesaj_ScpTVA", "dataInceputTvaInc", \
                  "dataSfarsitTvaInc", "dataActualizareTvaInc", "dataPublicareTvaInc", "tipActTvaInc", \
                  "statusTvaIncasare", "dataInactivare", "dataReactivare", "dataPublicare", "dataRadiere", \
                  "statusInactivi", "dataInceputSplitTVA", "dataAnulareSplitTVA", "statusSplitTVA"])


start = timeit.default_timer()
i = 0
for row in reader:
    i+=1
    cui = row[2]
    data_dict = {'cui': cui, 'data': date}
    data = []
    data.append(data_dict)
    r = requests.post(url, json=data)
    #print(r.text) # If you want to view entire content

    key_start = r.text.find('[')
    key_start+=1
    key_end = r.text.find(']')
    if r.status_code == 200:
        r_dict = r.text[key_start:key_end]
        r_dict = json.loads(r_dict)

        writer.writerow([row[0], row[1], row[2], r.status_code, data, \
                         r_dict['cui'], r_dict['data'], r_dict['denumire'], r_dict['adresa'], r_dict['scpTVA'], \
                         r_dict['data_inceput_ScpTVA'], r_dict['data_sfarsit_ScpTVA'], \
                         r_dict['data_anul_imp_ScpTVA'], r_dict['mesaj_ScpTVA'], r_dict['dataInceputTvaInc'], \
                         r_dict['dataSfarsitTvaInc'], r_dict['dataActualizareTvaInc'], \
                         r_dict['dataPublicareTvaInc'], r_dict['tipActTvaInc'], r_dict['statusTvaIncasare'], \
                         r_dict['dataInactivare'], r_dict['dataReactivare'], r_dict['dataPublicare'], \
                         r_dict['dataRadiere'], r_dict['statusInactivi'], r_dict['dataInceputSplitTVA'], \
                         r_dict['dataAnulareSplitTVA'], r_dict['statusSplitTVA']])


    else:
        writer.writerow ([row[0], row[1], row[2], r.status_code, data, \
                          "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", \
                          "", "", "", "", ""])       
# If you want to print results        
#    print([row[0], row[1], row[2], r.status_code, data, r_dict['denumire'], \
#          r_dict['cui'], r_dict['dataInceputSplitTVA'], r_dict['dataAnulareSplitTVA'], \
#          r_dict['statusSplitTVA']])
#    print(row[0])
        
    print('{} / {} / {}.'.format(row[0],r.status_code,i), end='\r')
    sys.stdout.flush()

file_upload.close()
file_save.close()

stop = timeit.default_timer()
print("request duration:", stop - start)

3.6.1 |Anaconda custom (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
request duration: 0.30942881602812206


In [5]:
"""
OUTPUT FILE IN CSV FORMAT

Result path_save: active_vendor.csv
Vendor_key,Vendor_name,CUI,status_code,data_set,cui,data,denumire,adresa,scpTVA,data_inceput_ScpTVA,data_sfarsit_ScpTVA,data_anul_imp_ScpTVA,mesaj_ScpTVA,dataInceputTvaInc,dataSfarsitTvaInc,dataActualizareTvaInc,dataPublicareTvaInc,tipActTvaInc,statusTvaIncasare,dataInactivare,dataReactivare,dataPublicare,dataRadiere,statusInactivi,dataInceputSplitTVA,dataAnulareSplitTVA,statusSplitTVA
SNN,S.N. NUCLEARELECTRICA S.A.,10874881,200,"[{'cui': '10874881', 'data': '2017-12-20'}]",10874881,2017-12-20,"SOCIETATEA NATIONALA ""NUCLEARELECTRICA"" SA","MUNICIPIUL BUCUREÅžTI, SECTOR 1, STR. POLONA, NR.65",True,1998-08-13, , ,platitor IN SCOPURI de TVA la data cautata,,,,,,False, , , , ,False,2017-09-29, ,True
SNP,OMV PETROM S.A.,1590082,200,"[{'cui': '1590082', 'data': '2017-12-20'}]",1590082,2017-12-20,OMV PETROM SA,"MUNICIPIUL BUCUREÅžTI, SECTOR 1, STR. CORALILOR, NR.22, BL.PETROM CITY",True,1993-07-01, , ,platitor IN SCOPURI de TVA la data cautata,,,,,,False, , , , ,False,,,False
"""

'\nOUTPUT FILE IN CSV FORMAT\n\nResult path_save: active_vendor.csv\nVendor_key,Vendor_name,CUI,status_code,data,denumire,cui:,dataInceputSplitTVA,dataAnulareSplitTVA,statusSplitTVA\nSNN,S.N. NUCLEARELECTRICA S.A.,10874881,200,"[{\'cui\': \'10874881\', \'data\': \'2017-12-20\'}]","SOCIETATEA NATIONALA ""NUCLEARELECTRICA"" SA",10874881,2017-09-29, ,True\nSNP,OMV PETROM S.A.,1590082,200,"[{\'cui\': \'1590082\', \'data\': \'2017-12-20\'}]",OMV PETROM SA,1590082,,,False\n'